In [1]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from visdom import Visdom

In [2]:
# 自定义数据集
class MyDataset(Dataset):
    def __init__(self, dataframe, root_dir, transform=None):
        self.questionnaire_data = dataframe
        self.root_dir = root_dir
        self.transform = transform
    def __len__(self):
        return len(self.questionnaire_data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir,"patient"+str(self.questionnaire_data.iloc[idx, 0]),self.questionnaire_data.iloc[idx, 1])
        image = io.imread(img_name)
        questionnaire = self.questionnaire_data.iloc[idx, 3:].tolist()
        label = self.questionnaire_data.iloc[idx, 2]
        sample = {'image': image, 'questionnaire': questionnaire, 'label': label}
        if self.transform:
            sample = self.transform(sample)
        return sample

In [3]:

class ToTensor(object):
    def __call__(self, sample):
        image = sample['image']
        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        image = image.transpose((2, 0, 1)).copy()
        return {'image': torch.from_numpy(image).float(), 'questionnaire': torch.Tensor(sample['questionnaire']), 'label':sample['label']}

In [4]:
class JointNet(nn.Module):
    def __init__(self,feature_extract=True, num_classes=3, hidden1=2048, hidden2=512, dropout=0.3):
        super(JointNet, self).__init__()
        model = models.vgg16(pretrained=True)
        self.features = model.features
        set_parameter_requires_grad(self.features, feature_extract)#固定特征提取层参数
        self.avgpool=model.avgpool
        self.hidden = nn.Sequential(
            nn.Linear(512*7*7 , hidden1),
            nn.ReLU(),
            nn.Linear(hidden1 , hidden2),
            nn.ReLU(),
        )
        self.classifier = nn.Sequential(
            nn.Linear(hidden2+37, hidden2+37),
            nn.Dropout(p=dropout),
            nn.ReLU(),
            nn.Linear(hidden2+37, num_classes)
        )

        
    def forward(self, x):
        img, quest = x
        img = self.features(img)
        img = self.avgpool(img)
        img = img.view(img.size(0), 512*7*7)
        img = self.hidden(img)
        joint = torch.cat([img, quest],1)
        out=self.classifier(joint)
        return out
    
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model=JointNet(feature_extract=False).to(device)
learning_rate=0.001
num_epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [6]:
model2 = model=JointNet(feature_extract=False).to(device)
model2.load_state_dict(torch.load("jointmodel.pth"))

<All keys matched successfully>

In [7]:
huawei_df = pd.read_csv("对照组huawei数据.csv", index_col=0)
huawei_dataset = MyDataset(huawei_df,"huawei",transform=ToTensor())
huawei_dataloader =DataLoader(huawei_dataset, batch_size=10,shuffle=True, num_workers=0)

In [8]:
model2.eval()
huawei_correct = 0
huawei_total = 0
with torch.no_grad():
    for sample_batched in huawei_dataloader:
        data = [sample_batched['image'].to(device),sample_batched['questionnaire'].to(device)]
        labels = sample_batched['label'].to(device)
        outputs = model2(data)
        _, predicted = torch.max(outputs.data, 1)
        huawei_total += labels.size(0)
        huawei_correct += (predicted == labels).sum().item()
print(huawei_correct/huawei_total)

1.0


In [9]:
iphone_df = pd.read_csv("对照组iphone数据.csv", index_col=0)
iphone_dataset = MyDataset(iphone_df,"iphone",transform=ToTensor())
iphone_dataloader =DataLoader(iphone_dataset, batch_size=10,shuffle=True, num_workers=0)

In [10]:
model2.eval()
iphone_correct = 0
iphone_total = 0
with torch.no_grad():
    for sample_batched in iphone_dataloader:
        data = [sample_batched['image'].to(device),sample_batched['questionnaire'].to(device)]
        labels = sample_batched['label'].to(device)
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        iphone_total += labels.size(0)
        iphone_correct += (predicted == labels).sum().item()
print(iphone_correct/iphone_total)

1.0


In [15]:
data_df = pd.read_csv("processed_data.csv", index_col=0)
train_df = data_df.sample(frac=0.8,random_state=0,axis=0)
test_df = data_df[~data_df.index.isin(train_df.index)]
train_dataset = MyDataset(train_df,"patient",transform=ToTensor())
test_dataset = MyDataset(test_df,"patient",transform=ToTensor())
train_dataloader =DataLoader(train_dataset, batch_size=10,shuffle=True, num_workers=0)
test_dataloader =DataLoader(test_dataset, batch_size=10,shuffle=True, num_workers=0)

In [33]:
train_prediction = []
Y_train = []
model2.eval()
with torch.no_grad():
    for sample_batched in train_dataloader:
        data = [sample_batched['image'].to(device),sample_batched['questionnaire'].to(device)]
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        train_prediction.extend(predicted.tolist())
        Y_train.extend(sample_batched['label'].tolist())


In [30]:
from sklearn.metrics import classification_report

In [39]:
print(classification_report(Y_train, train_prediction, target_names=['其他', '寻常型银屑病-轻度', '寻常型银屑病-中重度']))

              precision    recall  f1-score   support

          其他       0.88      1.00      0.93        78
   寻常型银屑病-轻度       1.00      0.86      0.92        78
  寻常型银屑病-中重度       1.00      1.00      1.00       286

    accuracy                           0.98       442
   macro avg       0.96      0.95      0.95       442
weighted avg       0.98      0.98      0.97       442



In [35]:
test_prediction = []
Y_test = []
model2.eval()
with torch.no_grad():
    for sample_batched in test_dataloader:
        data = [sample_batched['image'].to(device),sample_batched['questionnaire'].to(device)]
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        test_prediction.extend(predicted.tolist())
        Y_test.extend(sample_batched['label'].tolist())


In [38]:
print(classification_report(Y_test, test_prediction, target_names=['其他', '寻常型银屑病-轻度', '寻常型银屑病-中重度']))

              precision    recall  f1-score   support

          其他       0.95      1.00      0.98        21
   寻常型银屑病-轻度       1.00      0.92      0.96        13
  寻常型银屑病-中重度       1.00      1.00      1.00        76

    accuracy                           0.99       110
   macro avg       0.98      0.97      0.98       110
weighted avg       0.99      0.99      0.99       110

